In [4]:
# Initialization
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

# Constants
threshold = 1e-5 # In eV
iter = 100
Z = 2 # Helium

# Finite difference method setup
N = 1000
r_min = 0
r_max = 5
h = (r_max - r_min) / N
rho = np.arange(1,N+1)*h # Range from r_min to r_max

init_guess = 1/np.pi*Z**3*np.exp(-2*Z*rho) # Alex's guess

In [10]:
# Helper functions

def generate_wavefunction(rho,V,Z):
    # Determines the wavefunction and energy using the Hartree approximation

    A = np.diag(h**-2 - Z*rho**-1 + V,0) + \
        np.diag(-np.ones(N-1)/(2*h**2),1) + \
        np.diag(-np.ones(N-1)/(2*h**2),-1) # FDM matrix
    
    # Eigenvalues and -vectors
    (Eigenvalues,Eigenvectors) = np.linalg.eig(A)

    # For the ground state, we make use of the lowest eigenstate
    low = Eigenvectors[:,np.argmin(Eigenvalues)]
    #low = np.sign(low[0]) * low / np.linalg.norm(low) # Normalizing
    low = low / (np.sqrt(np.trapz(low**2,rho))*np.sign(low[0]))

    # Finally calculating the wavefunction using eq.34
    wf = low / (np.sqrt(4*np.pi)*rho)
    #wf = wf / np.trapz(wf,rho) # Normalization
    
    epsilon = np.min(Eigenvalues)

    return epsilon, wf

def generate_energy(epsilon, V_H, n, Z):
    # Excluding exchange-correlation terms
    E = Z*epsilon - 4*Z*np.pi* \
        np.trapz((V_H*n/2) * rho**2, rho)
    
    return E

def generate_hartree(rho,N,n):
    A = np.diag(-2*np.ones(N),0) + \
        np.diag(np.ones(N-1),1) + \
        np.diag(np.ones(N-1),-1)
    
    V = -4*np.pi*h**2*rho*n
    V[-1] -= 1.5
    
    V_H = (np.linalg.solve(A,V))*rho**-1
    return V_H

def self_consistency_solver(iterations,threshold):
    converged = False

    E = np.zeros(iter)
    n = init_guess
    V_H = Z*generate_hartree(rho,N,n)
    
    for i in range(iter):
        print(f'Iteration: {i}',end='\r')
        psi = generate_wavefunction(rho,V_H,Z)[1]
        epsilon = generate_wavefunction(rho,V_H,Z)[0]
        
        n = np.abs(psi)**2
        V_H = Z*generate_hartree(rho,N,n)
        E[i] = generate_energy(epsilon, V_H, n, Z)

        if np.abs(E[i]-E[i-1]) < threshold:
            print("E_g = " + str( E[i]))
            print("E = " + str(E[i]))
            converged = True
            break
            
    if converged == False:
        print(f'Did not converge in {iter} iterations.')

self_consistency_solver(iter,threshold)

KeyboardInterrupt: 